In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)


%matplotlib inline

hillstrom = pd.read_csv('/Users/Maksim/Desktop/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')
print(hillstrom.shape)
hillstrom.head()

(64000, 12)


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [2]:
dataset = hillstrom[hillstrom['segment']!='Mens E-Mail']
dataset.loc[:, 'treatment'] = dataset['segment'].map({
    'Womens E-Mail': 1,
    'No E-Mail': 0
})


dataset = dataset[dataset['channel']=='Phone']

dataset = dataset.drop(['segment', 'channel', 'conversion', 'spend'], axis=1)
print(dataset.shape)
dataset.head()

(18781, 9)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,recency,history_segment,history,mens,womens,zip_code,newbie,visit,treatment
0,10,2) $100 - $200,142.44,1,0,Surburban,0,0,1
5,6,2) $100 - $200,134.83,0,1,Surburban,0,1,1
6,9,3) $200 - $350,280.20,1,0,Surburban,1,0,1
7,9,1) $0 - $100,46.42,0,1,Urban,0,0,1
10,7,5) $500 - $750,548.91,0,1,Urban,1,1,1


In [3]:
from sklearn.model_selection import train_test_split

Xyt_tr, Xyt_val = train_test_split(dataset, test_size=0.5, random_state=42)

X_tr = Xyt_tr.drop(['visit', 'treatment'], axis=1)
y_tr = Xyt_tr['visit']
treat_tr = Xyt_tr['treatment']

X_val = Xyt_val.drop(['visit', 'treatment'], axis=1)
y_val = Xyt_val['visit']
treat_val = Xyt_val['treatment']

cat_cols = X_tr.select_dtypes(include='object').columns.tolist()

In [4]:
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder
from sklift.models import ClassTransformation
from xgboost import XGBClassifier

encoder = CatBoostEncoder(cols=cat_cols)
estimator = XGBClassifier(max_depth=2, random_state=42)
ct = ClassTransformation(estimator=estimator)

my_pipeline = Pipeline([
    ('encoder', encoder),
    ('model', ct)
])

In [5]:
my_pipeline.fit(
    X=X_tr,
    y=y_tr,
    model__treatment=treat_tr
)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/pipeline.py:354: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
         steps=[('encoder',
                 CatBoostEncoder(a=1, cols=['history_segment', 'zip_code'],
                                 drop_invariant=False, handle_missing='value',
                                 handle_unknown='value', random_state=None,
                                 return_df=True, sigma=None, verbose=0)),
                ('model',
                 ClassTransformation(estimator=XGBClassifier(base_score=0.5,
                                                             booster=None,
                                                             colsample_bylevel=1,
                                                             colsample_bynode=1,
                                                             colsample_by...
                                                             interaction_constraints=None,
                                                             learning_rate=0.300000012,
                                                

In [6]:
from sklift.metrics import uplift_at_k

uplift_predictions = my_pipeline.predict(X_val)

uplift_at_k(y_val, uplift_predictions, treat_val, strategy='overall')

0.04904281488369654